In [19]:
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar
from pytorch_lightning.strategies import ParallelStrategy
from pytorch_lightning.utilities.cli import LightningCLI

In [61]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

def _tokenize(x):

    x['input_ids'] = tokenizer.batch_encode_plus(
            x['text'], 
            add_special_tokens = True,
            max_length = 32,
            pad_to_max_length = True,)['input_ids']

    return x

def _prepare_ds(split):

    ds = load_dataset('rotten_tomatoes', split = split)
    ds = ds.shuffle(seed = 42)
    ds = ds.map(_tokenize, batched  =True)
    ds.set_format(type = 'torch', columns = ['input_ids', 'label'])

    return ds

train_ds, validation_ds, test_ds = map(_prepare_ds, ('train', 'validation', 'test'))

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (C:\Users\Yegyanathan V\.cache\huggingface\datasets\rotten_tomatoes_movie_review\default\1.0.0\40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
  0%|          | 0/9 [00:00<?, ?ba/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can gi

In [62]:
test_ds

Dataset({
    features: ['text', 'label', 'input_ids'],
    num_rows: 1066
})

In [63]:
validation_ds

Dataset({
    features: ['text', 'label', 'input_ids'],
    num_rows: 1066
})

In [65]:
test_ds['label']

tensor([1, 0, 1,  ..., 1, 0, 0])